In [1]:
import sys; sys.path.append(_dh[0].split("knowknow")[0])
from knowknow import *

In [2]:
showdocs("counter")

# Counting coocurrences

Cultural phenomena are rich in meaning and context. Moreover, the meaning and context are what we care about, so stripping that would be a disservice. "Consider Geertz:"
> Not only is the semantic structure of the figure a good deal more complex than it appears on the surface, but an analysis of that structure forces one into tracing a multiplicity of referential connections between it and social reality, so that the final picture is one of a configuration of dissimilar meanings out of whose interworking both the expressive power and the rhetorical force of the final symbol derive. (Geertz [1955] 1973, Chapter 8 Ideology as a Cultural System, p. 213)

The way people understanding their world shape their action, and understandings are heterogeneous in any community, woven into a complex web of interacting pieces and parts. Understandings are constantly evolving, shifting with every conversation or Breaking News. Any quantitative technique for studying meaning must be able to capture the relational structure of cultural objects, their temporal dynamics, or it cannot be meaning.

These considerations motivate how I have designed the data structure and code for this project. My attention to "cooccurrences" in what follows is an application of Levi Martin and Lee's (2018) formal approach to meaning. They develop the symbolic formalism I use below, as well as showing several general analytic strategies for inductive, ground-up meaning-making from count data. This approach is quite general, useful for many applications.

The process is rather simple, I count cooccurrences between various attributes. For each document, for each citation in that document, I increment a dozen counters, depending on attributes of the citation, paper, journal, or author. This counting process is done once, and can be used as a compressed form of the dataset for all further analyses. In the terminology of Levi Martin and Lee, I am constructing "hypergraphs", and I will use their notation in what follows. For example $[c*fy]$ indicates the dataset which maps from $(c, fy) \to count$.
$c$ is the name of the cited work. $fy$ is the publication year of the article which made the citation. $count$ is the number of citations which are at the intersection of these properties.

+ $[c]$ the number of citations each document receives
+ $[c*fj]$ the number of citations each document receives from each journal's articles
+ $[c*fy]$ the number of citations each document receives from each year's articles
+ $[fj]$ the number of citations from each journal
+ $[fj*fy]$ the number of citations in each journal in each year
+ $[t]$ cited term total counts
+ $[fy*t]$ cited term time series
+ term cooccurrence with citation and journal ($[c*t]$ and [fj*t]$)
+ "author" counts, the number of citations by each author ($[a]$ $[a*c]$ $[a*j*y]$)
+ [c*c]$, the cooccurrence network between citations
+ the death of citations can be studied using the $[c*fy]$ hypergraph
+ $[c*fj*t]$ could be used for analyzing differential associations of $c$ to $t$ across publication venues
+ $[ta*ta]$, $[fa*fa]$, $[t*t]$ and $[c*c]$ open the door to network-scientific methods



# References



+ Martin, John Levi, and Monica Lee. 2018. “A Formal Approach to Meaning.” Poetics 68(February):10–17.
+ Geertz, Clifford. 1973. The Interpretation of Cultures. New York: Basic Books, Inc.

# How to retrieve the requisite data

+ This notebook generates cooccurrence counts given Web of Science output.
+ No Web of Science output is included in knowknow in compliance with Web of Science terms of use.
+ **This notebook is only useful if you have your own Web of Science output**, and want to run analyses on that.

Data can be downloaded from any Web of Science search results page:
1. `Export -> Other File Formats`. 
2. In the dropdowns, specify `Record Content -> Full Record and Cited References` and `File Format -> Tab Delimited (Win)`
3. Type the folder containing the saved records as `.txt` files into the variable `wos_base` below.

# Instructions

1. Edit the parameters to fit your needs
2. Run all cells

# Parameters

In [1]:
#wos_base = "path/to/wos/data"
wos_base = "G:/My Drive/projects/qualitative analysis of literature/pre 5-12-2020/009 get everything from WOS"
journal_keep = ["ETHNIC AND RACIAL STUDIES", "LAW & SOCIETY REVIEW", "DISCOURSE & SOCIETY", "SOCIOLOGICAL INQUIRY", "CONTRIBUTIONS TO INDIAN SOCIOLOGY", "SOCIETY & NATURAL RESOURCES", "RATIONALITY AND SOCIETY", "DEVIANT BEHAVIOR", "ACTA SOCIOLOGICA", "SOCIOLOGY-THE JOURNAL OF THE BRITISH SOCIOLOGICAL ASSOCIATION", "WORK EMPLOYMENT AND SOCIETY", "SOCIOLOGICAL METHODS & RESEARCH", "SOCIOLOGICAL PERSPECTIVES", "JOURNAL OF MARRIAGE AND FAMILY", "WORK AND OCCUPATIONS", "JOURNAL OF CONTEMPORARY ETHNOGRAPHY", "THEORY AND SOCIETY", "POLITICS & SOCIETY", "SOCIOLOGICAL SPECTRUM", "RACE & CLASS", "ANTHROZOOS", "LEISURE SCIENCES", "COMPARATIVE STUDIES IN SOCIETY AND HISTORY", "SOCIAL SCIENCE QUARTERLY", "MEDIA CULTURE & SOCIETY", "SOCIOLOGY OF HEALTH & ILLNESS", "SOCIOLOGIA RURALIS", "SOCIOLOGICAL REVIEW", "TEACHING SOCIOLOGY", "BRITISH JOURNAL OF SOCIOLOGY", "JOURNAL OF THE HISTORY OF SEXUALITY", "SOCIOLOGY OF EDUCATION", "SOCIAL NETWORKS", "ARMED FORCES & SOCIETY", "YOUTH & SOCIETY", "POPULATION AND DEVELOPMENT REVIEW", "SOCIETY", "JOURNAL OF HISTORICAL SOCIOLOGY", "HUMAN ECOLOGY", "INTERNATIONAL SOCIOLOGY", "SOCIAL FORCES", "EUROPEAN SOCIOLOGICAL REVIEW", "JOURNAL OF HEALTH AND SOCIAL BEHAVIOR", "SOCIOLOGICAL THEORY", "SOCIAL INDICATORS RESEARCH", "POETICS", "HUMAN STUDIES", "SOCIOLOGICAL FORUM", "AMERICAN SOCIOLOGICAL REVIEW", "SOCIOLOGY OF SPORT JOURNAL", "SOCIOLOGY OF RELIGION", "JOURNAL OF LAW AND SOCIETY", "GENDER & SOCIETY", "BRITISH JOURNAL OF SOCIOLOGY OF EDUCATION", "LANGUAGE IN SOCIETY", "AMERICAN JOURNAL OF ECONOMICS AND SOCIOLOGY", "ANNALS OF TOURISM RESEARCH", "SOCIAL PROBLEMS", "INTERNATIONAL JOURNAL OF INTERCULTURAL RELATIONS", "SOCIAL SCIENCE RESEARCH", "SYMBOLIC INTERACTION", "JOURNAL OF LEISURE RESEARCH", "ECONOMY AND SOCIETY", "INTERNATIONAL JOURNAL OF COMPARATIVE SOCIOLOGY", "SOCIAL COMPASS", "SOCIOLOGICAL QUARTERLY", "JOURNAL OF MATHEMATICAL SOCIOLOGY", "AMERICAN JOURNAL OF SOCIOLOGY", "REVIEW OF RELIGIOUS RESEARCH", "RURAL SOCIOLOGY", "JOURNAL FOR THE SCIENTIFIC STUDY OF RELIGION", "ARCHIVES EUROPEENNES DE SOCIOLOGIE", "CANADIAN JOURNAL OF SOCIOLOGY-CAHIERS CANADIENS DE SOCIOLOGIE"]
name_blacklist = [
    "*us", 'Press', 'Knopf', '(January', 'Co', 'London', 'Bros', 'Books', 'Wilson','[anonymous]'
]

use_included_citations_filter = True
included_citations = load_variable("sociology-wos.included_citations")

debug = False
database_name = "sociology-wos"

RUN_PREGROUP = True

# First pass - just counting

In [3]:
from csv import DictReader

In [4]:
dcount = 0
total_inserts = 0
to_inserts = []
basedir = Path(wos_base)

# keeps track of all the years seen for grouped citations
multi_year = defaultdict(lambda: defaultdict(int))

In [5]:
# Instantiating counters
cnt_ind = defaultdict(lambda:defaultdict(int))
track_doc = defaultdict(lambda:defaultdict(set))
cnt_doc = defaultdict(lambda:defaultdict(int))

def cnt(term, space, doc):
    # it's a set, yo
    track_doc[space][term].add(doc)
    # update cnt_doc
    cnt_doc[space][term] = len(track_doc[space][term])
    # update ind count
    cnt_ind[space][term] += 1


In [6]:
# This cell ensures there are not overflow errors while importing large CSVs

import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [7]:
def fix_auths(auths):
    for a in auths:
        a = a.strip()
        if not len(a):
            continue

        a = a.split()[0].lower()
        yield a


def fix_refs(refs):
    for r in refs:
        yspl = re.split("((?:18|19|20)[0-9]{2})", r)

        if len(yspl) < 2:
            continue

        auth, year = yspl[:2]
        auth = auth.strip()
        year = int(year)

        if auth == "":
            continue

        auth = "&".join( fix_auths( auth.split(",") ) )

        yield ( auth, year, r )

# pre-group cited references while counting

In [8]:
groups=None

if RUN_PREGROUP:
    # loading precomputed groupings of cited books and articles, if the grouping has been generated

    try:
        groups = load_variable("%s.groups" % database_name)
        print("Groups successfully loaded.")

    except VariableNotFound:
        print("Groups don't exist yet. It's important to incorporate a fuzzy match for cited references."
              "Run this script once, then run 'trend summaries/cysum.ipynb' to generate summaries and filter out small cited works."
              "Then run 'grouping article and book names.ipynb' to generate groupings."
              "Then run this notebook again to generate counts for grouped entries."
              "And finally, to make sure cysum is up to date, run 'trend summaries/cysum.ipynb' one more time."
             )

    if groups is not None:
        from collections import defaultdict

        # find the most popular representation
        current_c = get_cnt("%s.doc"%database_name, ['c'])

        def get_reps(groups):
            ret = defaultdict(set)
            for k,v in groups.items():
                ret[v].add(k)
            ret = {
                k: max(v, key=lambda x:current_c['c'][x])
                for k,v in ret.items()
            }
            return ret

        group_reps = get_reps(groups)

Groups successfully loaded.
Loaded keys: dict_keys(['c'])
Available keys: ['c', 'c.fj', 'c.fy', 'c.fy.j', 'fa', 'fa.c', 'fa.fj', 'fa.fj.fy', 'fa.fy', 'fj', 'fj.fy', 'fj.ta', 'fj.ty', 'fy', 'fy.ta', 'fy.ty', 'ta', 'ty']


In [10]:
# processes WoS txt output files one by one, counting relevant cooccurrences as it goes

for i, f in enumerate( list(basedir.glob("**/*.txt")) ):

    with f.open(encoding='utf8') as pfile:
        r = DictReader(pfile, delimiter="\t")
        rows = list(r)

    if i % 50 == 0:
        print("File %s/%s: %s" % (i, len(list(basedir.glob("**/*.txt"))),f.name))

    for i, r in enumerate(rows):

        if r['DT'] != "Article":
            continue

        refs = r["CR"].strip().split(";")
        refs = list( fix_refs(refs) )

        if not len(refs):
            continue

        #print(refs)

        dcount += 1
        if dcount % 10000 == 0:
            print("Document: %s" % dcount)
            print("Citations: %s" % len(cnt_doc['c']))


        authors = r['AU'].split(";")
        authors = [x.strip() for x in authors]

        if False:
            for i in range(10):
                print("-"*20)


        uid = r['UT']


        try:
            int(r['PY'])
        except ValueError:
            continue



        for (auth,year,full_ref) in refs:
            
            # filter references based on cysum
            if use_included_citations_filter and ((auth,year) not in included_citations):
                continue
            
            if debug:
                print(full_ref)
            if 'DOI' not in full_ref and not len(re.findall(r', V[0-9]+, P[0-9]+', full_ref)):
                # splits off the author and year, and takes until the next comma
                full_ref = "|".join(
                    [auth]+
                    [x.strip().lower() for x in full_ref.split(",")[2:3]]
                )
            else:
                # just adds a fixed name and date to the front
                full_ref = "|".join(
                    [auth, str(year)] +
                    [",".join( full_ref.strip().lower().split(",")[2:] ).split(", doi")[0]]
                )
                
                
                
            # implement grouping of references
            if groups is not None:
                if full_ref in groups:
                    # retrieves retrospectively-computed groups
                    full_ref = group_reps[
                        groups[full_ref]
                    ]
                else:
                    # a small minority, the ones which are dropped in this process anyways
                    continue

                if debug:
                    print(full_ref)
                    print("--------------------")
                    
            
            auth = auth.lower()
            ref = (auth,year)
            
            cnt(auth, 'ta', uid)
            cnt((int(r['PY']),auth), 'fy.ta', uid)
            cnt((r['SO'],auth), 'fj.ta', uid)

            ref_str = "|".join(str(x) for x in ref)

            if ref[0] in name_blacklist:
                continue
            if "*" in ref[0]:
                continue
                
            if r['SO'] not in journal_keep:
                continue
                
            # BEGIN COUNTING!!!
            
            multi_year[full_ref][year] += 1
                
            cnt(r['SO'], 'fj', uid)
            cnt(int(r['PY']), 'fy', uid)
            cnt(ref[1], 'ty', uid)
            cnt((int(r['PY']), year), 'fy.ty', uid)
            cnt((r['SO'], year), 'fj.ty', uid)

            cnt((r['SO'],int(r['PY'])), 'fj.fy', uid)

            for a in authors:
                cnt(a, 'fa', uid)
                cnt((a,int(r['PY'])), 'fa.fy', uid)
                cnt((a,r['SO']), 'fa.fj', uid)
                cnt((a,r['SO'], int(r['PY'])), 'fa.fj.fy', uid)

                cnt((a,ref_str), 'fa.c', uid)
                
            
            cnt((a,ref_str), 'fa.c', uid)

            cnt(ref_str, 'c', uid)
            cnt((ref_str, int(r['PY'])), 'c.fy', uid)
            cnt((ref_str, r['SO']), 'c.fj', uid)
            cnt((ref_str, int(r['PY']), r['SO']), 'c.fy.j', uid)

File 0/386: 74501-75000.txt
Document: 10000
Citations: 85467
File 50/386: 3501-4000.txt
Document: 20000
Citations: 136655
Document: 30000
Citations: 163556
File 100/386: 28501-29000.txt
Document: 40000
Citations: 183078
File 150/386: 53501-54000.txt
Document: 50000
Citations: 198218
Document: 60000
Citations: 212387
File 200/386: 4501-5000.txt
Document: 70000
Citations: 231069
Document: 80000
Citations: 247630
File 250/386: 29501-30000.txt
Document: 90000
Citations: 263365
Document: 100000
Citations: 275960
File 300/386: 54501-55000.txt
Document: 110000
Citations: 284685
Document: 120000
Citations: 290066
File 350/386: 79501-80000.txt
Document: 130000
Citations: 294250


# cocitation counter

Because there are so many cited works, a full cocitation network would be prohibitively large to compute and store on disk. 
Furthermore, the full network is not very useful.
The following creates a cocitation network among only the most common 1000 cited works.
It counts the number of times `cnt['cc'][ (c1,c2) ]` that `c1` and `c2` are cited together in a work.
The `ind` and `doc` counters are identical for this counter, `'cc'`.

In [11]:
allowed_refs = Counter(dict(cnt_ind['c'].items())).most_common(1000)
allowed_refs = set( x[0] for x in allowed_refs )

print("# allowed references for cocitation analysis: %s" % len(allowed_refs))
print("Examples: %s" % str(list(allowed_refs)[:3]))

# enumerating cocitation for works with at least 10 citations
dcount = 0
refcount = 0

for i, f in enumerate(list(basedir.glob("**/*.txt"))):

    with f.open(encoding='utf8') as pfile:
        r = DictReader(pfile, delimiter="\t")
        rows = list(r)

    if i % 50 == 0:
        print("File %s/%s: %s" % (i, len(list(basedir.glob("**/*.txt"))), f))

    for i, r in enumerate(rows):

        if r['DT'] != "Article":
            continue

        refs = r["CR"].strip().split(";")
        refs = list(fix_refs(refs))

        if not len(refs):
            continue

        uid = r['UT']

        try:
            int(r['PY'])
        except ValueError:
            continue

        for ref in refs:

            ref_str = "|".join( str(x) for x in ref )
            if ref_str not in allowed_refs:
                continue

            for ref2 in refs:
                ref_str2 = "|".join( str(x) for x in ref2 )

                if ref_str2 <= ref_str:
                    continue
                if ref_str2 not in allowed_refs:
                    continue

                cnt((ref_str,ref_str2), 'cc', uid)
                refcount += 1
                if refcount % 10000 == 0:
                    print("%s cocitations logged" % refcount)
                    
print("Finished!")

# allowed references for cocitation analysis: 1000
Examples: ['collins|2000', 'chan|2007', 'johnson|1972']
File 0/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything from WOS\74501-75000.txt
File 50/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything from WOS\3501-4000.txt
File 100/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything from WOS\28501-29000.txt
File 150/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything from WOS\53501-54000.txt
File 200/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything from WOS\2006 and back\4501-5000.txt
File 250/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything from WOS\2006 and back\29501-30000.txt
File 300/386: G:\My Drive\projects\qualitative analysis of literature\pre 5-12-2020\009 get everything f

# trim counters, excluding cited works with only a single reference

Because of the huge number of cited works that only occur once, for efficiency it's best to simply log this statistic and eliminate them from the counter.


*This might not be necessary*

In [12]:
trimCounters = False

In [13]:
if trimCounters:

    onlyOne = len([x for x in cnt_doc['c'] if cnt_doc['c'][x] == 1])
    total = len(cnt_doc['c'])
    print("Of the %s cited works, %s (%0.2f%%) have only a single citation" % (
        total, onlyOne,
        100 * onlyOne/total
    ))

    terms = list(cnt_doc['c'].keys())
    counts = np.array([cnt_doc['c'][k] for k in terms])

    cutoff = 2
    to_remove = set([terms[int(i)] for i in np.argwhere(counts < cutoff)])
    print("consolidating", len(to_remove), list(to_remove)[:5])

    cnt_doc.keys()

    print("old size:", len(cnt_doc['c']))
    for tr in to_remove:
        del cnt_doc['c'][tr]
        del cnt_ind['c'][tr]
    print("new size:", len(cnt_doc['c']))

    print("old size:", len(cnt_doc['cy']))
    tydels = [x for x in cnt_doc['cy'] if x[0] in to_remove]
    for tydel in tydels:
        del cnt_doc['cy'][tydel]
        del cnt_ind['cy'][tydel]
    print("new size:", len(cnt_doc['cy']))

    print("old size:", len(cnt_doc['jc']))
    jtdels = [x for x in cnt_doc['jc'] if x[1] in to_remove]
    for jtdel in jtdels:
        del cnt_doc['jc'][jtdel]
        del cnt_ind['jc'][jtdel]
    print("new size:", len(cnt_doc['jc']))

In [ ]:
print(len(cnt_doc['c']), 'cited works')

# export generated variables

In [14]:
# retrieve and use the MOST COMMON pub date for each
pubyears = {
    k:max(s.keys(), key=lambda x:multi_year[k][x]) for k,s in multi_year.items()
    if len(s)
}
varname = "%s.pubyears"%database_name
save_variable(varname, pubyears)
print("saved %s" % varname)

saved sociology-wos.pubyears


In [15]:
save_cnt("%s.doc"%database_name, cnt_doc)

Saving sociology-wos.doc ___ ta
Saving sociology-wos.doc ___ fy.ta
Saving sociology-wos.doc ___ fj.ta
Saving sociology-wos.doc ___ fj
Saving sociology-wos.doc ___ fy
Saving sociology-wos.doc ___ ty
Saving sociology-wos.doc ___ fy.ty
Saving sociology-wos.doc ___ fj.ty
Saving sociology-wos.doc ___ fj.fy
Saving sociology-wos.doc ___ fa
Saving sociology-wos.doc ___ fa.fy
Saving sociology-wos.doc ___ fa.fj
Saving sociology-wos.doc ___ fa.fj.fy
Saving sociology-wos.doc ___ fa.c
Saving sociology-wos.doc ___ c
Saving sociology-wos.doc ___ c.fy
Saving sociology-wos.doc ___ c.fj
Saving sociology-wos.doc ___ c.fy.j


In [16]:
save_cnt("%s.ind"%database_name, cnt_ind)

Saving sociology-wos.ind ___ ta
Saving sociology-wos.ind ___ fy.ta
Saving sociology-wos.ind ___ fj.ta
Saving sociology-wos.ind ___ fj
Saving sociology-wos.ind ___ fy
Saving sociology-wos.ind ___ ty
Saving sociology-wos.ind ___ fy.ty
Saving sociology-wos.ind ___ fj.ty
Saving sociology-wos.ind ___ fj.fy
Saving sociology-wos.ind ___ fa
Saving sociology-wos.ind ___ fa.fy
Saving sociology-wos.ind ___ fa.fj
Saving sociology-wos.ind ___ fa.fj.fy
Saving sociology-wos.ind ___ fa.c
Saving sociology-wos.ind ___ c
Saving sociology-wos.ind ___ c.fy
Saving sociology-wos.ind ___ c.fj
Saving sociology-wos.ind ___ c.fy.j
